Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:

from corus import load_ods_gazeta path = 'gazeta.csv.gz' records = load_ods_gazeta(path) next(records)

что надо сделать:

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [1]:
import string
from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
import nltk
from nltk.tokenize import word_tokenize
import pickle
import numpy as np
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import matplotlib
import pandas as pd
%matplotlib inline
from string import punctuation
import re
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import tarfile
zipfile = 'gazeta_jsonl_v2_train.tar'
tar = tarfile.open(zipfile, 'r')
tar.extractall()
tar.close()

In [28]:
data = pd.read_json('gazeta_train.jsonl', lines = True)
data.head(5)

,date,url,summary,title,text
0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,Сегодня транспортный налог начисляется в завис...
1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",Словосочетание «музыкальный кинофестиваль» уже...
2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,После более чем 12-часовых консультаций Совет ...
3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,"Жертвами урагана «Агата», обрушившегося на Цен..."
4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,Решение ограничить рост тарифов естественных м...


In [29]:
sw = list(set(get_stop_words("ru")))

exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
  
    txt = str(txt)

    txt = "".join(c for c in txt if c not in exclude)
    
    txt = txt.lower()
    txt = re.sub('\s+', ' ', txt) # заменит пробельные символы на 1 пробел 
    txt = "".join(c for c in txt if c.isalpha() or c == ' ') # удалит все что не буква 
    
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw and len(word)>2]
    return " ".join(txt)

data['text'] = data['text'].apply(preprocess_text)

In [30]:
data.head(5)

,date,url,summary,title,text
0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,транспортный налог начисляться зависимость мощ...
1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",словосочетание музыкальный кинофестиваль звуча...
2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,часовой консультация совет безопасность оон со...
3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,жертва ураган агата обрушиться центральный аме...
4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,решение ограничить рост тариф естественный мон...


In [31]:
data.to_csv('data_csv_2') # csv весинт меньше сохраним в csv
df = pd.read_csv('data_csv_2')
df.head(5)

,Unnamed: 0,date,url,summary,title,text
0,0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,транспортный налог начисляться зависимость мощ...
1,1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",словосочетание музыкальный кинофестиваль звуча...
2,2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,часовой консультация совет безопасность оон со...
3,3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,жертва ураган агата обрушиться центральный аме...
4,4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,решение ограничить рост тариф естественный мон...


In [32]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

df['token'] = df['text'].apply(lambda x: word_tokenize(x))
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,date,url,summary,title,text,token
0,0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,транспортный налог начисляться зависимость мощ...,"[транспортный, налог, начисляться, зависимость..."
1,1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",словосочетание музыкальный кинофестиваль звуча...,"[словосочетание, музыкальный, кинофестиваль, з..."
2,2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,часовой консультация совет безопасность оон со...,"[часовой, консультация, совет, безопасность, о..."
3,3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,жертва ураган агата обрушиться центральный аме...,"[жертва, ураган, агата, обрушиться, центральны..."
4,4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,решение ограничить рост тариф естественный мон...,"[решение, ограничить, рост, тариф, естественны..."


In [33]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemm = WordNetLemmatizer()
df['lemm'] = df['token'].apply(lambda x: [lemm.lemmatize(w,'n') for w in x])
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Unnamed: 0,date,url,summary,title,text,token,lemm
0,0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,транспортный налог начисляться зависимость мощ...,"[транспортный, налог, начисляться, зависимость...","[транспортный, налог, начисляться, зависимость..."
1,1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",словосочетание музыкальный кинофестиваль звуча...,"[словосочетание, музыкальный, кинофестиваль, з...","[словосочетание, музыкальный, кинофестиваль, з..."
2,2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,часовой консультация совет безопасность оон со...,"[часовой, консультация, совет, безопасность, о...","[часовой, консультация, совет, безопасность, о..."
3,3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,жертва ураган агата обрушиться центральный аме...,"[жертва, ураган, агата, обрушиться, центральны...","[жертва, ураган, агата, обрушиться, центральны..."
4,4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,решение ограничить рост тариф естественный мон...,"[решение, ограничить, рост, тариф, естественны...","[решение, ограничить, рост, тариф, естественны..."


In [34]:
df.to_csv('data_lemm_csv') # cj[hfytybt]

In [36]:
df = pd.read_csv('data_lemm_csv')

In [37]:
sentences = df['lemm']
len(sentences[0])

3869

In [48]:
df.lemm

0        ['транспортный', 'налог', 'начисляться', 'зави...
1        ['словосочетание', 'музыкальный', 'кинофестива...
2        ['часовой', 'консультация', 'совет', 'безопасн...
3        ['жертва', 'ураган', 'агата', 'обрушиться', 'ц...
4        ['решение', 'ограничить', 'рост', 'тариф', 'ес...
                               ...                        
60959    ['сериал', 'ведьмак', 'самый', 'запрашивать', ...
60960    ['пилот', 'российский', 'авиакомпания', 'найти...
60961    ['свердловский', 'область', 'российский', 'вое...
60962    ['столица', 'ингушетия', 'магас', 'трое', 'зло...
60963    ['несмотря', 'приближение', 'новый', 'последни...
Name: lemm, Length: 60964, dtype: object

In [40]:
modelW2V = Word2Vec(sentences=sentences, vector_size=500, window=5, min_count=2, workers=7)
modelFT = FastText(sentences=sentences, vector_size=500, min_count=2, window=5, workers=7)

In [42]:
import annoy

In [38]:
pip install annoy ## установить висуал студио 14++ pip install pyproject.toml-based и cmake

  Using cached annoy-1.17.3.tar.gz (647 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for annoy: filename=annoy-1.17.3-cp39-cp39-win_amd64.whl size=52587 sha256=a3d6fcada08c4ef50544ea5117ad7591e299c49fbcf6d1342cf66e6660ff7e14
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\09\a9\54\37478e65995fe712f7da465749da9ddb21db6b1a599d591ac7
Successfully built annoy
Note: you may need to restart the kernel to use updated packages.


In [49]:
w2v_index = annoy.AnnoyIndex(500 ,'angular')
ft_index = annoy.AnnoyIndex(500 ,'angular')

index_map = {}
counter = 0

for line in df.lemm:
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = spls

        vector_w2v = np.zeros(500)
        vector_ft = np.zeros(500)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break


w2v_index.build(10)
ft_index.build(10)

True

In [58]:
def get_response(question, index, model, index_map):
    #question = preprocess_txt(question)
    vector = np.zeros(500)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [59]:
issure = 'футбол, иркутск, путин сша, коорелия '

In [61]:
a = get_response(issure, w2v_index, modelW2V, index_map)

In [67]:
df

,Unnamed: 0.1,Unnamed: 0,date,url,summary,title,text,token,lemm
0,0,0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,транспортный налог начисляться зависимость мощ...,"['транспортный', 'налог', 'начисляться', 'зави...","['транспортный', 'налог', 'начисляться', 'зави..."
1,1,1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",словосочетание музыкальный кинофестиваль звуча...,"['словосочетание', 'музыкальный', 'кинофестива...","['словосочетание', 'музыкальный', 'кинофестива..."
2,2,2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,часовой консультация совет безопасность оон со...,"['часовой', 'консультация', 'совет', 'безопасн...","['часовой', 'консультация', 'совет', 'безопасн..."
3,3,3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,жертва ураган агата обрушиться центральный аме...,"['жертва', 'ураган', 'агата', 'обрушиться', 'ц...","['жертва', 'ураган', 'агата', 'обрушиться', 'ц..."
4,4,4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,решение ограничить рост тариф естественный мон...,"['решение', 'ограничить', 'рост', 'тариф', 'ес...","['решение', 'ограничить', 'рост', 'тариф', 'ес..."
...,...,...,...,...,...,...,...,...,...
60959,60959,60959,2019-12-31 18:15:06,https://www.gazeta.ru/culture/2019/12/31/a_128...,«Ведьмак» от Netflix возглавил рейтинг самых в...,Малыш Йода не помог: Ведьмак обошел Мандалорца,сериал ведьмак самый запрашивать сериал соцсет...,"['сериал', 'ведьмак', 'самый', 'запрашивать', ...","['сериал', 'ведьмак', 'самый', 'запрашивать', ..."
60960,60960,60960,2019-12-31 18:51:06,https://www.gazeta.ru/social/2019/12/31/128937...,Тело российского пилота найдено на одном из пл...,Тело на пляже: как в Майами погиб российский п...,пилот российский авиакомпания найти мёртвый пл...,"['пилот', 'российский', 'авиакомпания', 'найти...","['пилот', 'российский', 'авиакомпания', 'найти..."
60961,60961,60961,2019-12-31 19:37:46,https://www.gazeta.ru/army/2019/12/31/12893804...,В Свердловской области прошли предварительные ...,«Машина будущего»: прошли испытания САУ «Коали...,свердловский область российский военный предва...,"['свердловский', 'область', 'российский', 'вое...","['свердловский', 'область', 'российский', 'вое..."
60962,60962,60962,2019-12-31 21:03:47,https://www.gazeta.ru/social/2019/12/31/128939...,"В столице Ингушетии на пост ДПС напали, по пре...",«Великий цинизм»: что известно о перестрелке в...,столица ингушетия магас трое злоумышленник нап...,"['столица', 'ингушетия', 'магас', 'трое', 'зло...","['столица', 'ингушетия', 'магас', 'трое', 'зло..."


In [71]:
data_row = data = pd.read_json('gazeta_train.jsonl', lines = True)
df['text2'] = data_row['text']

,date,url,summary,title,text
0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,Сегодня транспортный налог начисляется в завис...
1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",Словосочетание «музыкальный кинофестиваль» уже...
2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,После более чем 12-часовых консультаций Совет ...
3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,"Жертвами урагана «Агата», обрушившегося на Цен..."
4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,Решение ограничить рост тарифов естественных м...


In [82]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 120)
pd.set_option('display.width', 100)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('styler.latex.multicol_align',  'naive-l')
pd.set_option('display.colheader_justify', 'left')

In [101]:
df_v = pd.DataFrame(columns=df.columns)
for el in range(5):
    temp = df.loc[df['lemm']==a[el]]
    df_v = pd.concat([df_v, temp], axis=0, )
    #print(type(temp))
df_v

,Unnamed: 0.1,Unnamed: 0,date,url,summary,title,text,token,lemm,text2
3190,3190,3190,2010-10-20 10:41:51,https://www.gazeta.ru/business/2010/10/20/3429969.shtml,"Экспортные пошлины на нефтепродукты могут быть подняты до уровня 85–90% от нефтяной. Минфин предлагает это, чтобы не...",Россия забирает мазутный пряник,экспортный пошлина нефтепродукт существенно повысить сообщить журналист заместитель министр финансы сергей шаталов в...,"['экспортный', 'пошлина', 'нефтепродукт', 'существенно', 'повысить', 'сообщить', 'журналист', 'заместитель', 'минист...","['экспортный', 'пошлина', 'нефтепродукт', 'существенно', 'повысить', 'сообщить', 'журналист', 'заместитель', 'минист...","Экспортные пошлины на нефтепродукты могут быть существенно повышены уже с 2011 года, сообщил журналистам заместитель..."
38545,38545,38545,2018-04-23 20:22:48,https://www.gazeta.ru/army/vko/2018/04/23/11726419.shtml,Производитель продукции военного назначения «Алмаз-Антей» примет участие в выставке информационных и коммуникационны...,«Алмаз-Антей» примет участие в выставке «Связь-2018»,концерн вко алмазантея принять участие юбилейный международный выставка информационный коммуникационный технология с...,"['концерн', 'вко', 'алмазантея', 'принять', 'участие', 'юбилейный', 'международный', 'выставка', 'информационный', '...","['концерн', 'вко', 'алмазантея', 'принять', 'участие', 'юбилейный', 'международный', 'выставка', 'информационный', '...",Концерн ВКО Алмаз-Антей» примет участие в 30-й юбилейной международной выставке информационных и коммуникационных те...
51813,51813,51813,2019-05-16 18:20:12,https://www.gazeta.ru/science/2019/05/16_a_12359095.shtml,Главный конструктор режимного космического предприятия заявил об утечке в интернет секретных данных о российских спу...,«Это я оговорился»: кто раскрыл секреты Роскосмоса,интернет открытый доступ появиться дать российский космический аппарат создание запуск анонсироваться компанииразраб...,"['интернет', 'открытый', 'доступ', 'появиться', 'дать', 'российский', 'космический', 'аппарат', 'создание', 'запуск'...","['интернет', 'открытый', 'доступ', 'появиться', 'дать', 'российский', 'космический', 'аппарат', 'создание', 'запуск'...","В интернете в открытом доступе появились данные о российских космических аппаратах, создание и запуск которых не ано..."
50066,50066,50066,2019-04-01 22:26:33,https://www.gazeta.ru/business/2019/04/01/12278827.shtml,"Белоруссия не будет сертифицировать запрещенные санкционные продукты, отказавшись от поставок в Россию. Соответствую...",Запретный плод: Минск попался на санкционных яблоках,белоруссия прекратить сертификация плодоовощной санкционный продукция поступать страна государство предназначаться д...,"['белоруссия', 'прекратить', 'сертификация', 'плодоовощной', 'санкционный', 'продукция', 'поступать', 'страна', 'гос...","['белоруссия', 'прекратить', 'сертификация', 'плодоовощной', 'санкционный', 'продукция', 'поступать', 'страна', 'гос...","Белоруссия прекратила сертификацию плодоовощной «санкционной» продукции, которая поступала в эту страну из других го..."
6315,6315,6315,2011-02-28 19:04:48,https://www.gazeta.ru/financial/2011/02/28/3540393.shtml,"Госкорпорации необходимы для модернизации российской экономики, пришли к выводу аналитики правительства: нет смысла ...",Госкорпорации нужны модернизации,проблема госкорпорация произойти поворот приватизация реформирование ситуация стать плохой заявить директор институт...,"['проблема', 'госкорпорация', 'произойти', 'поворот', 'приватизация', 'реформирование', 'ситуация', 'стать', 'плохой...","['проблема', 'госкорпорация', 'произойти', 'поворот', 'приватизация', 'реформирование', 'ситуация', 'стать', 'плохой...","«При всех проблемах, которые есть у госкорпораций, если произойдет их поворот к приватизации или реформированию, то ..."


In [105]:
df_v[['text','title']]

,text,title
3190,экспортный пошлина нефтепродукт существенно повысить сообщить журналист заместитель министр финансы сергей шаталов в...,Россия забирает мазутный пряник
38545,концерн вко алмазантея принять участие юбилейный международный выставка информационный коммуникационный технология с...,«Алмаз-Антей» примет участие в выставке «Связь-2018»
51813,интернет открытый доступ появиться дать российский космический аппарат создание запуск анонсироваться компанииразраб...,«Это я оговорился»: кто раскрыл секреты Роскосмоса
50066,белоруссия прекратить сертификация плодоовощной санкционный продукция поступать страна государство предназначаться д...,Запретный плод: Минск попался на санкционных яблоках
6315,проблема госкорпорация произойти поворот приватизация реформирование ситуация стать плохой заявить директор институт...,Госкорпорации нужны модернизации
